In [ ]:
# Install the required packages
!pip3 install -U elasticsearch
!pip3 install -U jsonlines

     |████████████████████████████████| 225kB 2.7MB/s 


In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import time
import jsonlines
import pandas as pd

In [ ]:
# you can use RFC-1738 to specify the url | Initialize the ES client 
host = "https://search-covidtweets-pc7dqt6roqlapjsdifs6fk6oy4.us-east-1.es.amazonaws.com/"

es = Elasticsearch(['https://search-covidtweets-pc7dqt6roqlapjsdifs6fk6oy4.us-east-1.es.amazonaws.com/'], sniff_on_connection_fail= True) #, timeout=500000) << Not certain these parameters add value

In [ ]:
# Specify the search query via the json & execute the command 
# This is sample to create a baseline 
body = {
    "from": 0, # index from which the search is carried out
    "size":100, # Number of matches that should be returned
    "_source": ["created_at", "user.screen_name","user.location", "full_text"],
    "query" : {
        "constant_score" : {
            "filter" : {
                "bool": {
                    "must_not": {"term": {"user.location": ''}}
                }
            }
        }
    
}
}

q1 = es.search(index='covidtweets*', body=body)


In [ ]:
pd.DataFrame(q1['hits']['hits'])

,_index,_type,_id,_score,_source
0,covidtweets,_doc,4960977802362444979686349219675985584355120556...,1.0,{'full_text': 'RT @wsfa12news: UPDATE: Alabama...
1,covidtweets,_doc,4960977802362444979686349219676106476937082019...,1.0,{'full_text': '@DmKasganj @CMOfficeUP जय हिंद ...
2,covidtweets,_doc,4960977802362444979686349219676227369519043482...,1.0,{'full_text': 'RT @meiccymru: Already bored of...
3,covidtweets,_doc,4960977802362444979686349219678282543412388351...,1.0,{'full_text': 'RT @AyinAzirinee: Good article ...
4,covidtweets,_doc,4960977802362444979686349219678524328576311277...,1.0,{'full_text': 'RT @SupJvse: What Latino parent...
...,...,...,...,...,...
95,covidtweets,_doc,4960977802362444979686349219812110631643727822...,1.0,{'full_text': 'RT @CNN: This is how the Wuhan ...
96,covidtweets,_doc,4960977802362444979686349219812352416807650741...,1.0,{'full_text': 'RT @Doranimated: US and British...
97,covidtweets,_doc,4960977802362444979686349219812956879717458055...,1.0,"{'full_text': 'RT @aravosis: Yep.', 'created_a..."
98,covidtweets,_doc,4960977802362444979686349219813198664881380981...,1.0,{'full_text': 'RT @joncoopertweets: The corona...


In [ ]:
# Specify the search query via the json & execute the command
body = {
    "from": 0, # index from which the search is carried out
    "size":1000, # Number of matches that should be returned
    "_source": ["created_at", "user.screen_name","user.location", "user.description", "full_text"],
    "query": {
    "query_string": {
      "query": "(advocaat) OR (staatssecretaris) OR (manager) OR (gezondheidscentrum) OR (medisch) OR (welzijnsinstelling) OR (agent) OR (apoth)"
      
    }
  }
    
      }

q1 = es.search(index='dutch*', body=body)
query_DF = pd.DataFrame(q1['hits']['hits'])

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

The next few code blocks I will try to implement a scroll API to extract all the docs from an index 

Source:
https://kb.objectrocket.com/elasticsearch/how-to-use-python-to-make-scroll-queries-to-get-all-documents-in-an-elasticsearch-index-752

In [ ]:
# import the Elasticsearch client library
from elasticsearch import Elasticsearch, exceptions

# import JSON and time
import json, time

index = "dutch*"

In [ ]:
# declare a filter query dict object
match_all = {
    "size": 5000,
    "_source": ["created_at", "user.screen_name","user.location", "user.description", "full_text"],
    "query": {
        "match_all": {}
        }
    }

# make a search() request to get all docs in the index
resp = es.search(
            index = 'dutch*',
            body = match_all,
            scroll = '15s' # length of time to keep search context
        )

# keep track of pass scroll _id
old_scroll_id = resp['_scroll_id']

In [ ]:
allData = pd.DataFrame()

In [ ]:
azaaaaaAAQA
time.sleep(5)

# use a 'while' iterator to loop over document 'hits'
while len(resp['hits']['hits']):
            query_DF = pd.DataFrame(resp['hits']['hits'])
            queryDF = pd.DataFrame(list(query_DF['_source']))
            user_info = queryDF['user'].apply(pd.Series)
            queryDF = pd.concat( [queryDF.drop(['user'],axis=1) , user_info], axis=1)
            allData = pd.concat([allData, queryDF], axis =0)
            # make a request using the Scroll API
            resp = es.scroll(
                scroll_id = old_scroll_id,
                scroll = '15s' # length of time to keep search context
            )

            # check if there's a new scroll ID
            if old_scroll_id != resp['_scroll_id']:
                print ("NEW SCROLL ID:", resp['_scroll_id'])

            # keep track of pass scroll _id
            old_scroll_id = resp['_scroll_id']

            # print the response results
            print ("\nresponse for index:", index)
            print ("_scroll_id:", resp['_scroll_id'])
            print ('Collected data:', str(allData.shape[0]))
            time.sleep(5)
            

NotFoundError: ignored

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
allData.shape

(201721, 5)

In [ ]:
allData.head()

,full_text,created_at,screen_name,description,location
0,"@pflegearzt @Friedelkorn @LAguja44 Pardon, wol...",Mon Mar 09 12:26:29 +0000 2020,TheoRettich,"I ❤️science, therefore a Commie. ☭ FALGSC: P...",Netherlands
1,RT @grantshapps: Aviation demand is reduced du...,Mon Mar 09 12:26:34 +0000 2020,davidiwanow,I tweet a lot but love to engage & converse. P...,"Amsterdam, The Netherlands"
2,RT @DDStandaard: De droom van D66 wordt werkel...,Mon Mar 09 12:26:37 +0000 2020,EricL65,,
3,De droom van D66 wordt werkelijkheid: COVID-19...,Mon Mar 09 12:26:47 +0000 2020,EhrErwin,Budget-Life Coach. Time management Coaching. b...,Amsterdam
4,RT @eucopresident: Following consultations I w...,Mon Mar 09 12:27:12 +0000 2020,TomasRiemens,Politiek redacteur Hart van Nederland,Amsterdam


In [ ]:
queryDF = pd.DataFrame(list(query_DF['_source']))

In [ ]:
user_info = queryDF['user'].apply(pd.Series)

In [ ]:
queryDF = pd.concat( [queryDF.drop(['user'],axis=1) , user_info], axis=1)

In [ ]:
queryDF.head(15)

,full_text,created_at,screen_name,description,location
0,"@pflegearzt @Friedelkorn @LAguja44 Pardon, wol...",Mon Mar 09 12:26:29 +0000 2020,TheoRettich,"I ❤️science, therefore a Commie. ☭ FALGSC: P...",Netherlands
1,RT @grantshapps: Aviation demand is reduced du...,Mon Mar 09 12:26:34 +0000 2020,davidiwanow,I tweet a lot but love to engage & converse. P...,"Amsterdam, The Netherlands"
2,RT @DDStandaard: De droom van D66 wordt werkel...,Mon Mar 09 12:26:37 +0000 2020,EricL65,,
3,De droom van D66 wordt werkelijkheid: COVID-19...,Mon Mar 09 12:26:47 +0000 2020,EhrErwin,Budget-Life Coach. Time management Coaching. b...,Amsterdam
4,RT @eucopresident: Following consultations I w...,Mon Mar 09 12:27:12 +0000 2020,TomasRiemens,Politiek redacteur Hart van Nederland,Amsterdam
5,Read absorb and share https://t.co/IHPDY6qS4w,Mon Mar 09 12:27:17 +0000 2020,DenisonChapman,British European Optimist Scientist Engineer. ...,The Netherlands
6,RT @FrankvanWijck: Het aantal deskundigen over...,Mon Mar 09 12:27:35 +0000 2020,martijnmollers,Chirurg @ Heelkunde Friesland | Auteur Hart en...,
7,RT @HanTenBroeke: Ook zo benieuwd of het Coron...,Mon Mar 09 12:27:36 +0000 2020,jterHeide,"Militair, KL, VBDD, 1GNC, NATO, EW/Cyber Polit...",NLD-DEU
8,Het wordt dommer en dommer! We weten nu alleen...,Mon Mar 09 12:27:52 +0000 2020,Barbes_NL,❌ A critical error occurred trying to install ...,"Amsterdam, The Netherlands"
9,@volkskrant We hebben toch niet een ander viru...,Mon Mar 09 12:27:56 +0000 2020,ninjasavant,I'm only here to piss people off so the little...,"Amsterdam, The Netherlands"


In [ ]:
queryDF.tail(15)

,full_text,created_at,screen_name,description,location
985,Steeds meer bedrijven blunderen met hun corona...,Thu Jul 30 12:58:50 +0000 2020,Wikram,"Interim manager in changing environment, Direc...","The Hague, Netherlands"
986,Corona test came in and it's negative 🥳,Thu Aug 20 08:57:11 +0000 2020,hennesheijmans,20 | Manager @ATL_Letshe | Team & Sponsorship ...,"Nijmegen, Netherlands"
987,Everyone please wear a damn mask so we can go ...,Sun Jul 26 23:47:38 +0000 2020,JoeyLeiz,"Free Agent. Former Premier League, Champions L...","Amsterdam, The Netherlands"
988,RT @jorisvancauter: Goed dat het parlement op ...,Thu Jun 18 20:22:36 +0000 2020,DevolderArne,,
989,RT @frankvanardenne: “Tussen 17 maart tot 21 m...,Thu Jun 25 06:03:33 +0000 2020,noade_leon,Advocaat | tuchtrecht | (jeugd)strafrecht |fou...,
990,@VogelvrijeHArts Er is bij zes medewerkers van...,Sun Sep 13 17:31:18 +0000 2020,_j3lena_,#prayfortheNetherlands\n\n#covid19nl\n#herdimm...,Universe
991,RT @RutgervdNoort: Leestip:” Medisch gezien is...,Mon May 04 11:26:17 +0000 2020,AvdBn,Het leven is kut. Daarom ben ik voor abortus.,Nederland / The Netherlands
992,RT @LudwigBollaerts: Zullen we deze foto morge...,Tue Mar 17 09:59:25 +0000 2020,AMPOORTERHANS,"Verba volent,scripta manent",
993,"RT @EricChrSmit: Eelco van Anken, celbioloog e...",Thu Mar 19 06:29:30 +0000 2020,Dontlookawayor1,GAB: sniperonwatch,"Usan, Scotland"
994,RT @Bloempje141: Deze meneer met verstand en e...,Thu Aug 20 08:56:40 +0000 2020,WBV01789100,#Samen staan we Sterk\n#WWG1WGA\n#AntiGates\n#...,


In [ ]:
allData.to_csv('dutch_tweets.csv', index=False)

In [ ]:
!pip3 install -U spacy

In [ ]:
!python3 -m spacy download nl_core_news_sm

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("nl_core_news_sm")

In [ ]:
doc = nlp("Dit is een zin.")

In [ ]:
print([(w.text, w.pos_) for w in doc])

[('Dit', 'PRON'), ('is', 'VERB'), ('een', 'DET'), ('zin', 'NOUN'), ('.', 'SYM')]


In [ ]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Dit dit PRON VNW|aanw|pron|stan|vol|3o|ev__Person=3|PronType=Dem nsubj Xxx True True
is zijn VERB WW|pv|tgw|ev__Number=Sing|Tense=Pres|VerbForm=Fin cop xx True True
een een DET LID|onbep|stan|agr__Definite=Ind det xxx True True
zin zin NOUN N|soort|ev|basis|zijd|stan__Gender=Com|Number=Sing ROOT xxx True False
. . SYM LET punct . False False


In [ ]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

In [ ]:
def checkWords(textString):
  matches = ['advocaat', 'staatssecretaris', 'gezondheidscentrum', 'welzijnsinstelling', 'medisch']

  if any(x in textString for x in matches):
    return True
  else:
    return False


In [ ]:
queryDF['Industry'] = queryDF['full_text'].apply(checkWords)

In [ ]:
sum(queryDF['Industry'])

167

In [ ]:
queryDF.head(15)

,full_text,created_at,screen_name,description,location,Industry
0,RT @NVALeuven: Onaanvaardbaar! 😡😡😡\n\nSpoedig ...,Mon Sep 14 14:08:15 +0000 2020,karel_mortier,Vader | Echtgenoot | Leuvenaar | Advocaat arb...,,False
1,Coronabonus voor leiding Zaans Medisch Centrum...,Sat Aug 08 13:55:42 +0000 2020,rightsofearth,Agent provocateur. Stelt rare dingen #stemrech...,Amsterdam Zuidoost,False
2,Dubbele verantwoordelijkheid dus voor werkgeve...,Wed Sep 16 09:06:56 +0000 2020,wisbin,𝔻𝕠𝕔𝕖𝕟𝕥 𝕧𝕠/𝕊𝕠𝕔𝕚𝕠𝕝𝕠𝕠𝕘/𝔸𝕟𝕥𝕚𝕥𝕙𝕖ï𝕤𝕥/𝔻𝕒𝕥𝕒 ℝ𝕒𝕡𝕡𝕖𝕣/ℙ𝕚𝕩...,"Schiedam, Nederland",False
3,RT @Elsdijkstra95: Nee tatuurlijk niet he. Waa...,Wed Aug 19 15:27:21 +0000 2020,lorelei3112,Hannelore Bruijn Manus-je-van-alles in apothe...,Streefkerk,False
4,RT @mikercarpenter: News that got completely b...,Wed May 13 14:49:54 +0000 2020,mvanderKist,"Jurist, oud-advocaat, kinderrechten, (jeugd)st...",Apeldoorn e.o. Netherlands,False
5,"RT @FaQuix: Nog zo’n willekeur, de kleurencode...",Sun Aug 02 21:23:26 +0000 2020,WNiemegeers,Advocaat - Адвокат/юрист - Voorzitter “Vlaams ...,,False
6,RT @RubenKorthorst: De ‘willing to pay’ lijkt ...,Wed Mar 25 11:46:27 +0000 2020,Lammerthoeve,"Huisarts, Opleider, Bestuurslid VPHuisartsen. ...",,False
7,Zij die vinden dat covid19 een voldoende concr...,Sun Jul 26 05:58:50 +0000 2020,2Plot,West-Vlaams advocaat wiens mening niet door ie...,,False
8,RT @FleurAgemaPVV: Ingezet voor de zorg? Je be...,Wed Jun 24 18:29:59 +0000 2020,PalingRobert,"Senior redacteur Vastgoedmarkt, eeuwige advoca...",Dordrecht,False
9,RT @bslagter: (A) Van Dissel legt in die video...,Wed Sep 23 06:38:09 +0000 2020,RubenKorthorst,uroloog | medisch manager | @ziekenhuisMST | U...,,False


In [ ]:
queryDF.to_excel('query.xlsx', index=False)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

This code block will calculate the weekday/ month/ year dat points 

In [ ]:
import datetime

allData['dateObject'] = allData['created_at'].apply(lambda x: datetime.datetime.strptime(x, '%a %b %d %H:%M:%S %z %Y'))

allData['Day'] = allData['dateObject'].apply(lambda x: x.day)
allData['Weekday'] = allData['dateObject'].apply(lambda x: x.weekday)
allData['Month'] = allData['dateObject'].apply(lambda x: x.month)
allData['Year'] = allData['dateObject'].apply(lambda x: x.year)
allData.drop(['dateObject'], axis=1, inplace=True)

In [ ]:
allData.head()

,full_text,created_at,screen_name,description,location,Weekday,Month,Year,Day
0,"@pflegearzt @Friedelkorn @LAguja44 Pardon, wol...",Mon Mar 09 12:26:29 +0000 2020,TheoRettich,"I ❤️science, therefore a Commie. ☭ FALGSC: P...",Netherlands,0,3,2020,9
1,RT @grantshapps: Aviation demand is reduced du...,Mon Mar 09 12:26:34 +0000 2020,davidiwanow,I tweet a lot but love to engage & converse. P...,"Amsterdam, The Netherlands",0,3,2020,9
2,RT @DDStandaard: De droom van D66 wordt werkel...,Mon Mar 09 12:26:37 +0000 2020,EricL65,,,0,3,2020,9
3,De droom van D66 wordt werkelijkheid: COVID-19...,Mon Mar 09 12:26:47 +0000 2020,EhrErwin,Budget-Life Coach. Time management Coaching. b...,Amsterdam,0,3,2020,9
4,RT @eucopresident: Following consultations I w...,Mon Mar 09 12:27:12 +0000 2020,TomasRiemens,Politiek redacteur Hart van Nederland,Amsterdam,0,3,2020,9


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

The next code block will use the geoCoder API (OSM) to get the lat,long for each of the location (unique) in the dataset

In [ ]:
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [ ]:
locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


In [ ]:
# Get all unique locations 

allLocation = allData['location'].unique().tolist()
print("Nuber of unique locations",len(allLocation))

Nuber of unique locations 15283


In [ ]:
allLocation =pd.DataFrame(allLocation)
print(allLocation.shape)
allLocation.columns = ['location']
print(allLocation.head())

(15283, 1)
                     location
0                 Netherlands
1  Amsterdam, The Netherlands
2                            
3                   Amsterdam
4             The Netherlands


I have a one second rate limiter so for 15283 records it may take a lot of time! 42hrs+ 

So lets try to break up the records into smaller units & run them in parallel

In [ ]:
import numpy as np

allLocationArray = np.array_split(allLocation, 10)
allLocationArray[1]

,location
1529,Veere
1530,overal en nergens
1531,"Earth,The One and Only Truth"
1532,"Diepenveen, Netherlands"
1533,Oosterhout
...,...
3053,"Leiden, Den Haag, Teylingen"
3054,"Liskwartier, Rotterdam"
3055,Scheveningen ..
3056,Het Heuge Neurden


In [ ]:
for idx in range(len(allLocationArray)):
  allLocationArray[idx].to_excel("allLocation_chunk"+str(idx)+".xlsx", index=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/"My Drive"/"Dutch Tweets/allLocation_chunk8.xlsx" .

cp: cannot stat 'drive/My Drive/Dutch Tweets/allLocation_chunk8.xlsx': No such file or directory


In [ ]:
!ls drive/"My Drive"/"Dutch Tweets"

allLocation_chunk0.csv	allLocation_chunk4.csv	allLocation_chunk8.csv
allLocation_chunk1.csv	allLocation_chunk5.csv	allLocation_chunk9.csv
allLocation_chunk2.csv	allLocation_chunk6.csv	dutch_tweets.csv
allLocation_chunk3.csv	allLocation_chunk7.csv


In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [ ]:
allDataList = np.array_split(allData, 10)

In [ ]:
for idx in range(len(allDataList)):
  print(idx)
  try:
    allDataList[idx].to_csv("allDataList_chunk"+str(idx)+".csv", index=True)
  except:
    print("ERR in idx: ", idx)

0
1
2
3
4
5
6
7
8
9


In [ ]:
!cp allDataList*.csv drive/"My Drive"/"Dutch Tweets"
!ls drive/"My Drive"/"Dutch Tweets"

allDataList_chunk0.csv	allDataList_chunk8.csv	   allLocation_chunk5.csv
allDataList_chunk1.csv	allDataList_chunk9.csv	   allLocation_chunk6.csv
allDataList_chunk2.csv	allDataList_chunk9.gsheet  allLocation_chunk7.csv
allDataList_chunk3.csv	allLocation_chunk0.csv	   allLocation_chunk8.csv
allDataList_chunk4.csv	allLocation_chunk1.csv	   allLocation_chunk9.csv
allDataList_chunk5.csv	allLocation_chunk2.csv	   dutch_tweets.csv
allDataList_chunk6.csv	allLocation_chunk3.csv
allDataList_chunk7.csv	allLocation_chunk4.csv


In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [ ]:
# Read location chunks 
import pandas as pd 

idx = 8

allLocation_chunk = pd.read_excel("allLocation_chunk"+str(idx)+".xlsx")
allLocation_chunk.shape
allLocation_chunk.head()

,Unnamed: 0,location
0,12227,"Renkum, The Netherlands"
1,12228,Noord-Holland en Zuid-Holland
2,12229,Sin city / Amsterdam
3,12230,"Heeswijk-Dinther, Bernheze"
4,12231,"Hintham-Zuid, 's-Hertogenbosch"


In [ ]:
# 2- - create location column
allLocation_chunk['point_info'] = allLocation_chunk['location'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('N.v.t.',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1368, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1328, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1356, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/u

In [ ]:
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
allLocation_chunk['point'] = allLocation_chunk['point_info'].apply(lambda loc: tuple(loc.point) if loc else None)

allLocation_chunk.head()

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude,Province
0,12227,"Renkum, The Netherlands","(Renkum, Gelderland, Nederland, (51.9751065, 5...","(51.9751065, 5.7335691, 0.0)",51.975107,5.733569,0.0,Gelderland
1,12228,Noord-Holland en Zuid-Holland,"(Noord, Rotterdam, Zuid-Holland, Nederland, (5...","(51.9330231, 4.4722707, 0.0)",51.933023,4.472271,0.0,Zuid-Holland
2,12229,Sin city / Amsterdam,"(Amsterdam, Lomas del Sol, Culiacán Rosales, C...","(24.8452918, -107.3821699, 0.0)",24.845292,-107.382170,0.0,False
3,12230,"Heeswijk-Dinther, Bernheze","(Heeswijk-Dinther, Bernheze, Noord-Brabant, Ne...","(51.664083399999996, 5.466634823340177, 0.0)",51.664083,5.466635,0.0,Noord-Brabant
4,12231,"Hintham-Zuid, 's-Hertogenbosch","(Hintham-zuid, Rosmalen, Hintham, Noord-Braban...","(51.6976614, 5.343617582664436, 0.0)",51.697661,5.343618,0.0,Noord-Brabant


In [ ]:
# 4 - split point column into latitude, longitude and altitude columns
allLocation_chunk[['latitude', 'longitude', 'altitude']] = pd.DataFrame(allLocation_chunk['point'].tolist(), index=allLocation_chunk.index)

allLocation_chunk.head(15)

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude,Province
0,12227,"Renkum, The Netherlands","(Renkum, Gelderland, Nederland, (51.9751065, 5...","(51.9751065, 5.7335691, 0.0)",51.975107,5.733569,0.0,Gelderland
1,12228,Noord-Holland en Zuid-Holland,"(Noord, Rotterdam, Zuid-Holland, Nederland, (5...","(51.9330231, 4.4722707, 0.0)",51.933023,4.472271,0.0,Zuid-Holland
2,12229,Sin city / Amsterdam,"(Amsterdam, Lomas del Sol, Culiacán Rosales, C...","(24.8452918, -107.3821699, 0.0)",24.845292,-107.382170,0.0,False
3,12230,"Heeswijk-Dinther, Bernheze","(Heeswijk-Dinther, Bernheze, Noord-Brabant, Ne...","(51.664083399999996, 5.466634823340177, 0.0)",51.664083,5.466635,0.0,Noord-Brabant
4,12231,"Hintham-Zuid, 's-Hertogenbosch","(Hintham-zuid, Rosmalen, Hintham, Noord-Braban...","(51.6976614, 5.343617582664436, 0.0)",51.697661,5.343618,0.0,Noord-Brabant
5,12232,"Grunobuurt, Groningen","(Grunobuurt, Groningen, Nederland, (53.2066154...","(53.2066154, 6.5597569, 0.0)",53.206615,6.559757,0.0,Groningen
6,12233,"Lochem-Zuid, Lochem",None,None,NaN,NaN,NaN,False
7,12234,"ÜT: 36.098565,-80.311023",None,None,NaN,NaN,NaN,False
8,12235,BTS |﹙G﹚I﹣ᴅʟᴇ | TOP 🇵🇹,None,None,NaN,NaN,NaN,False
9,12236,La Capitale,"(La Capitale, Valensole, Forcalquier, Alpes-de...","(43.8713728, 5.8995945, 0.0)",43.871373,5.899595,0.0,False


In [ ]:
!cp drive/"My Drive"/"Dutch Tweets"/*.geojson .

In [ ]:
!pip install geopandas
!pip install geopy


In [ ]:
import geopandas as gpd

provinces = gpd.read_file('provinces.geojson')

print(provinces.shape)
provinces.head()

(12, 4)


,name,regioFacetId,level,geometry
0,Drenthe,tcm:106-353397-1024,3,"POLYGON ((6.41328 52.98552, 6.36252 53.03397, ..."
1,Flevoland,tcm:106-353410-1024,3,"POLYGON ((5.36115 52.67573, 5.37726 52.76481, ..."
2,Friesland (Fryslân),tcm:106-353417-1024,3,"POLYGON ((5.08707 53.32307, 5.10178 53.36803, ..."
3,Gelderland,tcm:106-353445-1024,3,"POLYGON ((5.60602 51.99416, 5.59200 52.00139, ..."
4,Groningen,tcm:106-353502-1024,3,"POLYGON ((6.28698 53.34138, 6.27368 53.34527, ..."


In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
def checkProvince(p1):
  #print(len(p1))
  if p1 != None and len(p1) >= 2 :
    p1 = Point(p1[1], p1[0])  
    provinces['InProvince'] = provinces['geometry'].apply(lambda x: x.contains(p1))
    selProvince = provinces[provinces['InProvince'] == True]
    #print(selProvince)
    if selProvince.shape[0] > 0:
      return selProvince['name'].values[0]
    else:
      return False
  else:
    return False

In [ ]:
allLocation_chunk['Province'] = allLocation_chunk['point'].apply(lambda x: checkProvince(x))
allLocation_chunk.head(15)

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude,Province
0,12227,"Renkum, The Netherlands","(Renkum, Gelderland, Nederland, (51.9751065, 5...","(51.9751065, 5.7335691, 0.0)",51.975107,5.733569,0.0,Gelderland
1,12228,Noord-Holland en Zuid-Holland,"(Noord, Rotterdam, Zuid-Holland, Nederland, (5...","(51.9330231, 4.4722707, 0.0)",51.933023,4.472271,0.0,Zuid-Holland
2,12229,Sin city / Amsterdam,"(Amsterdam, Lomas del Sol, Culiacán Rosales, C...","(24.8452918, -107.3821699, 0.0)",24.845292,-107.382170,0.0,False
3,12230,"Heeswijk-Dinther, Bernheze","(Heeswijk-Dinther, Bernheze, Noord-Brabant, Ne...","(51.664083399999996, 5.466634823340177, 0.0)",51.664083,5.466635,0.0,Noord-Brabant
4,12231,"Hintham-Zuid, 's-Hertogenbosch","(Hintham-zuid, Rosmalen, Hintham, Noord-Braban...","(51.6976614, 5.343617582664436, 0.0)",51.697661,5.343618,0.0,Noord-Brabant
5,12232,"Grunobuurt, Groningen","(Grunobuurt, Groningen, Nederland, (53.2066154...","(53.2066154, 6.5597569, 0.0)",53.206615,6.559757,0.0,Groningen
6,12233,"Lochem-Zuid, Lochem",None,None,NaN,NaN,NaN,False
7,12234,"ÜT: 36.098565,-80.311023",None,None,NaN,NaN,NaN,False
8,12235,BTS |﹙G﹚I﹣ᴅʟᴇ | TOP 🇵🇹,None,None,NaN,NaN,NaN,False
9,12236,La Capitale,"(La Capitale, Valensole, Forcalquier, Alpes-de...","(43.8713728, 5.8995945, 0.0)",43.871373,5.899595,0.0,False


In [ ]:
# idx = 0
allLocation_chunk.to_csv("allLocation_chunk"+str(idx)+".csv", index=False)
print("Geo processing completed....allLocation_chunk"+str(idx) + ".csv")

Geo processing completed....allLocation_chunk8.csv


In [ ]:
!cp allLocation_chunk8.csv drive/"My Drive"/"Dutch Tweets"/allLocation_chunk8.csv

In [ ]:
temp = pd.read_csv("allLocation_chunk8.csv")